<a href="https://colab.research.google.com/github/blindauth/abstention_experiments/blob/master/catdog/AnalyzeCatDogResults.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!rm -r abstention
!pip uninstall abstention
!git clone https://github.com/blindauth/abstention.git
!pip install abstention/
![[ -e abstention_experiments ]] || git clone https://github.com/blindauth/abstention_experiments.git

rm: cannot remove 'abstention': No such file or directory
Cloning into 'abstention'...
remote: Enumerating objects: 127, done.
remote: Counting objects: 100% (127/127), done.
remote: Compressing objects: 100% (84/84), done.
remote: Total 127 (delta 75), reused 91 (delta 39), pack-reused 0
Receiving objects: 100% (127/127), 65.16 KiB | 1.03 MiB/s, done.
Resolving deltas: 100% (75/75), done.
Processing ./abstention
  Stored in directory: /tmp/pip-ephem-wheel-cache-wyeavu99/wheels/60/8e/d2/9e9ca02e7b5f76bfda2e2daa6dcbe42c19095c502ccb653729
Successfully built abstention
Cloning into 'abstention_experiments'...
remote: Enumerating objects: 1129, done.
remote: Counting objects: 100% (1129/1129), done.
remote: Compressing objects: 100% (1124/1124), done.
remote: Total 4067 (delta 51), reused 1048 (delta 5), pack-reused 2938
Receiving objects: 100% (4067/4067), 289.56 MiB | 39.98 MiB/s, done.
Resolving deltas: 100% (67/67), done.


In [0]:
!ls abstention_experiments/catdog

AnalyzeCatDogResults.ipynb	   record_5_model_3L3uV_preds
catdog_abstention_results.json.gz  record_6_model_jXjNL_preds
record_10_model_Xu18x_preds	   record_7_model_A5Cuf_preds
record_11_model_O6Kxa_preds	   record_9_model_QCnbs_preds
record_1_model_wqD9U_preds	   Run_CatDog_Experiments.ipynb
record_2_model_eDiQm_preds	   test_labels.txt.gz
record_3_model_Mwl4q_preds	   valid_labels.txt.gz
record_4_model_3Y6wQ_preds


In [0]:
import gzip
import json
catdog_abstention_results =\
  json.loads(gzip.open(
    "abstention_experiments/catdog/catdog_abstention_results.json.gz").read())

In [0]:
from abstention.figure_making_utils import (
    wilcox_srs, get_ustats_mat,
    get_tied_top_and_worst_methods)
import numpy as np
from collections import defaultdict

abstfracs = ['0.15', '0.3']

metric_to_abstfrac_to_methodnames = {
    'tpratfpr_0.01': {
        '0.15': ['distfrom0.5:None',
                 'distfrom0.5:EM',
                 'js:None',
                 'js:EM',
                 'mcwindowdelta_tpratfpr_15pc:None',
                 'mcwindowdelta_tpratfpr_15pc:EM',
                 'fumera_tpratfpr_15pc:None',
                 'fumera_tpratfpr_15pc:EM',
                 'uncertainty:None',
                 'uncertainty:EM'],
        '0.3': ['distfrom0.5:None',
                'distfrom0.5:EM',
                'js:None',
                'js:EM',
                'mcwindowdelta_tpratfpr_30pc:None',
                'mcwindowdelta_tpratfpr_30pc:EM',
                'fumera_tpratfpr_30pc:None',
                'fumera_tpratfpr_30pc:EM',
                'uncertainty:None',
                'uncertainty:EM']
    },
    'auprc': {
        '0.15': ['distfrom0.5:None',
                 'distfrom0.5:EM',
                 'js:None',
                 'js:EM',
                 'estwindowdelta_auprc_15pc:None',
                 'estwindowdelta_auprc_15pc:EM',
                 'fumera_auprc_15pc:None',
                 'fumera_auprc_15pc:EM',
                 'uncertainty:None',
                 'uncertainty:EM'],
        '0.3': ['distfrom0.5:None',
                'distfrom0.5:EM',
                'js:None',
                'js:EM',
                'estwindowdelta_auprc_30pc:None',
                'estwindowdelta_auprc_30pc:EM',
                'fumera_auprc_30pc:None',
                'fumera_auprc_30pc:EM',
                'uncertainty:None',
                'uncertainty:EM']
    }
}
    

nicemethodname_to_metric_to_adapt_to_abstfrac_to_methodname = {
    "Est $\Delta$Metric (Ours)": {
        'tpratfpr_0.01':{
            'yesadapt': {
                '0.3': 'mcwindowdelta_tpratfpr_30pc:EM',
                '0.15': 'mcwindowdelta_tpratfpr_15pc:EM'
            }, 'noadapt': {
                '0.3': 'mcwindowdelta_tpratfpr_30pc:None',
                '0.15': 'mcwindowdelta_tpratfpr_15pc:None'
            }
        },
        'auprc': {
            'yesadapt': {
                '0.3': 'estwindowdelta_auprc_30pc:EM',
                '0.15': 'estwindowdelta_auprc_15pc:EM'
            }, 'noadapt': {
                '0.3': 'estwindowdelta_auprc_30pc:None',
                '0.15': 'estwindowdelta_auprc_15pc:None'
            }
        }
    },
    "JS Div. (Ours)": {
        'tpratfpr_0.01': {
          'yesadapt': {'0.3': 'js:EM', '0.15': 'js:EM'},
          'noadapt': {'0.3': 'js:None', '0.15': 'js:None'}
        }, 'auprc': {
          'yesadapt': {'0.3': 'js:EM', '0.15': 'js:EM'},
          'noadapt': {'0.3': 'js:None', '0.15': 'js:None'}
        }
    },
    "Dist. from 0.5": {
        'tpratfpr_0.01': {
          'yesadapt': {'0.3': 'distfrom0.5:EM', '0.15': 'distfrom0.5:EM'},
          'noadapt': {'0.3': 'distfrom0.5:None', '0.15': 'distfrom0.5:None'},
        }, 'auprc': {
          'yesadapt': {'0.3': 'distfrom0.5:EM', '0.15': 'distfrom0.5:EM'},
          'noadapt': {'0.3': 'distfrom0.5:None', '0.15': 'distfrom0.5:None'},
        }
    },
    "Test-time Dropout": {
        'tpratfpr_0.01': {
          'yesadapt': {'0.3': 'uncertainty:EM', '0.15': 'uncertainty:EM'},
          'noadapt': {'0.3': 'uncertainty:None', '0.15': 'uncertainty:None'}
        }, 'auprc': {
          'yesadapt': {'0.3': 'uncertainty:EM', '0.15': 'uncertainty:EM'},
          'noadapt': {'0.3': 'uncertainty:None', '0.15': 'uncertainty:None'}
        }
    },
    "Fumera et al.": {
        'tpratfpr_0.01': {
          'yesadapt': {
                '0.3': 'fumera_tpratfpr_30pc:EM',
                '0.15': 'fumera_tpratfpr_15pc:EM'},
          'noadapt': {
                '0.3': 'fumera_tpratfpr_30pc:None',
                '0.15': 'fumera_tpratfpr_15pc:None'}
        }, 'auprc': {
          'yesadapt': {
                  '0.3': 'fumera_tpratfpr_30pc:EM',
                  '0.15': 'fumera_tpratfpr_15pc:EM'},
          'noadapt': {
                  '0.3': 'fumera_tpratfpr_30pc:None',
                  '0.15': 'fumera_tpratfpr_15pc:None'} 
        }
    }
}
nicemethodname_orders = [
    "Est $\Delta$Metric (Ours)",
    "JS Div. (Ours)",
    "Dist. from 0.5",
    "Fumera et al.",
    "Test-time Dropout"]

def stdderr(vals):
  return np.std(vals, ddof=1)/np.sqrt(len(vals))

#identify the best methods
metricname_to_abstfrac_to_bestmethods = defaultdict(dict)  
for metricname in ['tpratfpr_0.01', 'auprc']:
  for abstfrac in abstfracs:
    methodnames = metric_to_abstfrac_to_methodnames[metricname][abstfrac]
    nsamples = len(list(
      catdog_abstention_results['metricname_to_abstfrac_to_abstnadapt_to_perf']
                               [metricname][abstfrac].values())[0])
    method_to_perfs = (catdog_abstention_results['metricname_to_abstfrac_to_abstnadapt_to_perf']
                                   [metricname][abstfrac])
    ustats_mat = get_ustats_mat(
        method_to_perfs=method_to_perfs,
        method_names=methodnames,
        max_ustat=nsamples*(nsamples+1)/2)
    tied_top_methods, tied_worst_methods =(
        get_tied_top_and_worst_methods(
            ustats_mat,
            methodnames,
            #0.05 threshold for one-sided test when N=30 is 159,
            #0.025 threshold is 137
            #0.005 threshold is 109
            #from https://www.oreilly.com/library/view/nonparametric-statistics-a/9781118840429/bapp02.xhtml
            threshold=137
        ))
    tied_top_methods = [methodnames[x] for x in tied_top_methods]
    print("bestmethods",metricname, abstfrac, tied_top_methods)
    metricname_to_abstfrac_to_bestmethods[metricname][abstfrac] =\
      set(tied_top_methods)
    
    
thetable = """
\\begin{table*}[!h]
\\tiny
\\begin{center}
\\begin{tabular}{ | c c | c c c | }
\\hline
\\multicolumn{2}{| c |}{ } & \\multicolumn{3}{ c |}{Cat v Dog Sensitivity @ 99\\% Specificity}\\\\
\\multicolumn{1}{| c}{Method} & \\multicolumn{1}{c |}{Adapted?} & Base & @30\\% Abst. & @15\\% Abst.\\\\\\hline
"""

for nicemethodname in nicemethodname_orders:
  for adapt in ['yesadapt', 'noadapt']:
    if adapt in nicemethodname_to_metric_to_adapt_to_abstfrac_to_methodname[nicemethodname]['auprc']:
      thetable += nicemethodname
      thetable += " & "+("Y" if adapt=='yesadapt' else 'N')
      for metricname in ['tpratfpr_0.01']:
        metricname_to_adaptname_to_baselineperfs =\
          catdog_abstention_results['metricname_to_adaptname_to_baselineperfs']
        metricname_to_abstfrac_to_abstnadapt_to_perf =\
          catdog_abstention_results['metricname_to_abstfrac_to_abstnadapt_to_perf']
        baseperfs = metricname_to_adaptname_to_baselineperfs[metricname][
                     ('EM' if adapt=='yesadapt' else 'None')]
        thetable += (" & "+str(np.round(np.mean(baseperfs),4))
                     +"$\\pm$"+str(np.round(stdderr(baseperfs),4)))
        for abstfrac in ['0.3', '0.15']:
          methodname =\
           nicemethodname_to_metric_to_adapt_to_abstfrac_to_methodname[
            nicemethodname][metricname][adapt][abstfrac]
          isbest = (methodname in
            metricname_to_abstfrac_to_bestmethods[metricname][abstfrac])
          methodperfs = metricname_to_abstfrac_to_abstnadapt_to_perf[
            metricname][abstfrac][methodname]
          thetable += " & "
          if (isbest):
            thetable += "\\textbf{"
          thetable += (str(np.round(np.mean(methodperfs),4))
                       +"$\pm$"+str(np.round(stdderr(methodperfs),4)))
          if (isbest):
            thetable += "}"
      thetable += "\\\\ \\hline\n"
      #if (nicemethodname=="JS Div. (Ours)" and adapt=='noadapt'):
      #    thetable += "\\hline\n"

thetable += """\\end{tabular}
\\end{center}
\\caption{\\textbf{}. }
\\label{tab:catdog}
\\end{table*}
"""
print("\n\n"+thetable)

bestmethods tpratfpr_0.01 0.15 ['mcwindowdelta_tpratfpr_15pc:EM']
bestmethods tpratfpr_0.01 0.3 ['mcwindowdelta_tpratfpr_30pc:EM', 'fumera_tpratfpr_30pc:None']
bestmethods auprc 0.15 ['distfrom0.5:None', 'js:EM']
bestmethods auprc 0.3 ['distfrom0.5:None']



\begin{table*}[!h]
\tiny
\begin{center}
\begin{tabular}{ | c c | c c c | }
\hline
\multicolumn{2}{| c |}{ } & \multicolumn{3}{ c |}{Cat v Dog Sensitivity @ 99\% Specificity}\\
\multicolumn{1}{| c}{Method} & \multicolumn{1}{c |}{Adapted?} & Base & @30\% Abst. & @15\% Abst.\\\hline
Est $\Delta$Metric (Ours) & Y & 0.3318$\pm$0.0061 & \textbf{0.6135$\pm$0.0114} & \textbf{0.4598$\pm$0.0082}\\ \hline
Est $\Delta$Metric (Ours) & N & 0.3318$\pm$0.0061 & 0.5616$\pm$0.0102 & 0.4438$\pm$0.0084\\ \hline
JS Div. (Ours) & Y & 0.3318$\pm$0.0061 & 0.404$\pm$0.0079 & 0.3655$\pm$0.0064\\ \hline
JS Div. (Ours) & N & 0.3318$\pm$0.0061 & 0.3687$\pm$0.0073 & 0.3466$\pm$0.0065\\ \hline
Dist. from 0.5 & Y & 0.3318$\pm$0.0061 & 0.5226$\pm$0.01 & 0.4122$\pm